# Import lib


In [ ]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import seaborn as sns
import matplotlib.pyplot as plt
import re
from sklearn.preprocessing import MinMaxScaler



# Read data

In [ ]:
# df = pd.read_csv('../data.csv')
df = pd.read_csv('../data.csv')

In [ ]:
#check null
null_counts = df.isnull().sum()
print(null_counts)

# Discover and visualise the data

#### remove irrelevant features

In [ ]:
remove_col = [
    'id',
    'member_id',
    'emp_title',
    'issue_d',
    'desc',
    'title',
    'zip_code',
    'addr_state',
    'earliest_cr_line',
    'last_pymnt_d',
    'last_pymnt_amnt',
    'next_pymnt_d',
    'last_credit_pull_d',
    'collections_12_mths_ex_med',
    'mths_since_last_major_derog',
    'policy_code',

    # Vu Anh
    # 'home-ownership'
    # 'purpose', 
    # 'sub_grade',

    'tot_coll_amt', # This can be removed 
    'tot_cur_bal', 
    'open_acc_6m', # Remove because all null value got defautl = 1
    'open_il_6m', 

    # Darrel
    # 'open_il_12m', 
    # 'open_il_24m', 
    # 'mths_since_rcnt_il', 
    # 'total_bal_il', 
    # 'il_util', 
    # 'open_rv_12m' ,
    # 'open_rv_24m', 

    # Vanness
    'max_bal_bc', 
    'all_util', 
    'total_rev_hi_lim', 
    'inq_fi', 
    'total_cu_tl', 
    'inq_last_12m'
]

## Proportion of default = 1 and default = 0 

In [ ]:
# Get the value counts of 'default_ind'
value_counts = df['default_ind'].value_counts()

# Calculate the proportions
proportion_0 = value_counts[0] / len(df)
proportion_1 = value_counts[1] / len(df)

print(f"Proportion of 0: {proportion_0 * len(df)}")
print(f"Proportion of 1: {proportion_1 * len(df)}")




## Handle home-ownership

In [ ]:
# Remove rows with value ANY

df = df[df['home_ownership'] != 'ANY']

df['home_ownership'].unique()

In [ ]:
# Level encoding for home ownership 

home_type = ['RENT', 'OWN', 'MORTGAGE', 'OTHER', 'NONE']  # Unique values for encoding

# Create an instance of LabelEncoder
encoder = LabelEncoder()

# Fit the encoder on the grades
encoder.fit(home_type)

# Encode the 'grade' column in the DataFrame
df['home_ownership'] = encoder.transform(df['home_ownership'])

## Handle purpose

In [ ]:
df['purpose'].unique()

purposes = ['credit_card', 'car', 'small_business', 'other', 'wedding',
       'debt_consolidation', 'home_improvement', 'major_purchase',
       'medical', 'moving', 'vacation', 'house', 'renewable_energy',
       'educational']

# Create an instance of LabelEncoder
encoder = LabelEncoder()

# Fit the encoder on the grades
encoder.fit(purposes)
# Encode the 'grade' column in the DataFrame
df['purpose'] = encoder.transform(df['purpose'])


In [ ]:
# Count the occurrences of each label
label_counts = df['purpose'].value_counts().sort_index()

# Print the label counts

'''
for label, count in label_counts.items():
    print(f"{label}: {count}")
'''

## Handle sub_grade

The sub-grade is a more detailed classification that further divides borrowers within each grade. It typically includes additional factors such as credit history, employment stability, and debt-to-income ratio. This additional level of detail can help in distinguishing the risk profile of borrowers within the same grade.

This is why we should keep the subgrade

In [ ]:
df['sub_grade'].unique()

# TODO: check if we should use data binning for this attribute


To make best use of level encoding, we sort the sub-grade by order to present the relationship between subgrades

In [ ]:
subgrades = ['B2', 'C4', 'C5', 'C1', 'B5', 'A4', 'E1', 'F2', 'C3', 'B1', 'D1',
       'A1', 'B3', 'B4', 'C2', 'D2', 'A3', 'A5', 'D5', 'A2', 'E4', 'D3',
       'D4', 'F3', 'E3', 'F4', 'F1', 'E5', 'G4', 'E2', 'G3', 'G2', 'G1',
       'F5', 'G5']

def custom_sort_key(subgrade):
    match = re.match(r'([A-Za-z]+)(\d+)', subgrade)
    letter = match.group(1)
    number = int(match.group(2))
    
    return letter, number

sorted_subgrades = sorted(subgrades, key=custom_sort_key)

print(sorted_subgrades)

In [ ]:
# Level encoding for sorted sub-grade 

# Create an instance of LabelEncoder
encoder = LabelEncoder()

# Fit the encoder on the grades
encoder.fit(sorted_subgrades)

# Encode the 'grade' column in the DataFrame
df['sub_grade'] = encoder.transform(df['sub_grade'])

In [ ]:
df['sub_grade'].unique()

In [ ]:
df = df.drop(remove_col, axis=1)

In [ ]:
np.shape(df)

In [ ]:
df.info()

### Process missing value

In [ ]:
# Display all attributes with missing values

null_columns = df.columns[df.isnull().any()] 
null_columns_result = df.isnull().any()[null_columns] 
null_columns_result

#### Emp_length

In [ ]:
df['emp_length'].unique()

In [ ]:
employment_lengths = ['< 1 year', '1 year', '2 years', '3 years', '4 years', '5 years', '6 years', '7 years', '8 years', '9 years', '10+ years', 'nan']

label_encoder = LabelEncoder()

# Fit the LabelEncoder with unique values
label_encoder.fit(employment_lengths)

# Encode the attribute values
df['emp_length'] = label_encoder.transform(df['emp_length'])

In [ ]:
df['emp_length'].unique()

#### mths_since_last_delinq 

In [ ]:
df['mths_since_last_delinq'] = df['mths_since_last_delinq'].fillna(-1)

#### mths_since_last_record

In [ ]:
df['mths_since_last_record'] = df['mths_since_last_record'].fillna(-1)

#### revol_util

In [ ]:
imputer = SimpleImputer(strategy='mean')

revol_util = df['revol_util'].values.reshape(-1,1)

revol_util_imputed = imputer.fit_transform(revol_util)

df['revol_util'] = revol_util_imputed

#### annual_inc_joint

In [ ]:
# aggregate annual_inc and annual_inc_joint
df.loc[df['application_type'] == 'JOINT', 'annual_inc'] = df.loc[df['application_type'] == 'JOINT', 'annual_inc_joint']
df = df.drop('annual_inc_joint', axis=1)

#### dti_joint 

In [ ]:
df.loc[df['application_type'] == 'JOINT', 'dti'] = df.loc[df['application_type'] == 'JOINT', 'dti_joint']
df = df.drop('dti_joint', axis=1)

#### verification_status_joint 

In [ ]:
df.loc[df['application_type'] == 'JOINT', 'verification_status'] = df.loc[df['application_type'] == 'JOINT', 'verification_status_joint']
df = df.drop('verification_status_joint', axis=1)

### Data binning

#### term

In [ ]:
df['term'].unique()

In [ ]:
term = [' 36 months', ' 60 months']  # Unique values for encoding

# Create an instance of LabelEncoder
encoder = LabelEncoder()

# Fit the encoder on the grades
encoder.fit(term)

# Encode the 'grade' column in the DataFrame
df['term'] = encoder.transform(df['term'])

### grade

In [ ]:
grades = ['A', 'B', 'C', 'D', 'E', 'F', 'G']  # Unique values for encoding

encoder = LabelEncoder()

encoder.fit(grades)

df['grade'] = encoder.transform(df['grade'])

### verification_status

In [ ]:
veri = ['Not Verified', 'Source Verified', 'Verified']  # Unique values for encoding

encoder = LabelEncoder()

encoder.fit(veri)


df['verification_status'] = encoder.transform(df['verification_status'])

### pymnt_plan

In [ ]:
plan = ['n', 'y']  # Unique values for encoding

encoder = LabelEncoder()

encoder.fit(plan)

df['pymnt_plan'] = encoder.transform(df['pymnt_plan'])

### application_type

In [ ]:
type = ['INDIVIDUAL', 'JOINT'] # Unique values for encoding

encoder = LabelEncoder()

encoder.fit(type)

df['application_type'] = encoder.transform(df['application_type'])

#### initial_list_status

In [ ]:

status = ['f', 'w'] # Unique values for encoding

encoder = LabelEncoder()

encoder.fit(status)

df['initial_list_status'] = encoder.transform(df['initial_list_status'])

### Cleaning of the following

    'open_il_12m', 842681 missing values, can consider scrapping as there's no value
    'open_il_24m', 842681 missing values, can consider scrapping as there's no value
    'mths_since_rcnt_il', 843035 missing values, while there's alot of missing values however, those recorded >1 months since most recent instalment account opened will 100% not default
    'total_bal_il', 842681 missing values, while there's alot of missing values however, those recorded >0 total current balance of all instalment account will 100% not default
    'il_util', 844360 missing values, 
    'open_rv_12m' , 842681 missing values, 
    'open_rv_24m', 842681 missing values, 

 'open_il_24m' & 'open_il_24m'

In [ ]:
#check null
null_counts = df.isnull().sum()
print(null_counts)

In [ ]:
# Before Inputation
columns_to_visualize = ['open_il_12m', 'open_il_24m']

# Plotting settings
plt.figure(figsize=(10, 6))
plt.title("Frequency of Imputed Values")
plt.xlabel("Counts")
plt.ylabel("Frequency")

# Create count plots for each column
for column in columns_to_visualize:
    sns.countplot(data=df, x=column)
    
plt.legend(columns_to_visualize)
plt.show()

In [ ]:
df['open_il_12m'].unique()
df[['open_il_12m', 'open_il_24m']].describe()


In [ ]:
# Inpute with 0

columns_to_impute = ['open_il_12m', 'open_il_24m']

# Impute missing values with 0
df[columns_to_impute] = df[columns_to_impute].fillna(0)


In [ ]:
# After Inputation
columns_to_visualize = ['open_il_12m', 'open_il_24m']

# Plotting settings
plt.figure(figsize=(10, 6))
plt.title("Frequency of Imputed Values")
plt.xlabel("Counts")
plt.ylabel("Frequency")

# Create count plots for each column
for column in columns_to_visualize:
    sns.countplot(data=df, x=column)
    
plt.legend(columns_to_visualize)
plt.show()

'mths_since_rcnt_il'

In [ ]:
df['mths_since_rcnt_il'].unique()

In [ ]:
# Before Inputation
columns_to_visualize = ['mths_since_rcnt_il']

# Plotting settings
plt.figure(figsize=(10, 6))
plt.title("Frequency of Imputed Values")
plt.xlabel("Counts")
plt.ylabel("Frequency")

# Create count plots for each column
for column in columns_to_visualize:
    sns.countplot(data=df, x=column)
    
plt.legend(columns_to_visualize)
plt.show()

In [ ]:
# Check against default_ind
condition = (df['mths_since_rcnt_il'] > 0) & (df['default_ind'] == 1)
result_df = df[condition]

result_df

In [ ]:
# Inpute with 0

columns_to_impute = ['mths_since_rcnt_il']


# Impute missing values with 0
df[columns_to_impute] = df[columns_to_impute].fillna(0)

In [ ]:
# After Inputation
columns_to_visualize = ['mths_since_rcnt_il']

# Plotting settings
plt.figure(figsize=(10, 6))
plt.title("Frequency of Imputed Values")
plt.xlabel("Counts")
plt.ylabel("Frequency")

# Create count plots for each column
for column in columns_to_visualize:
    sns.countplot(data=df, x=column)
    
plt.legend(columns_to_visualize)
plt.show()

total_bal_il

In [ ]:
filtered_df = df.dropna(subset=['total_bal_il'])
total_bal_il_values = filtered_df['total_bal_il'].unique()

# Display unique values and the rest of the columns
filtered_df


In [ ]:
# Check against default_ind
condition = (df['total_bal_il'] > 0) & (df['default_ind'] == 1)
result_df = df[condition]

result_df

In [ ]:
# Before Inputation
columns_to_visualize = ['total_bal_il']

# Plotting settings
plt.figure(figsize=(10, 6))
plt.title("Frequency of Imputed Values")
plt.xlabel("Counts")
plt.ylabel("Frequency")

# Create count plots for each column
for column in columns_to_visualize:
    sns.countplot(data=df, x=column)
    
plt.legend(columns_to_visualize)
plt.show()

In [ ]:
# Inpute with 0

columns_to_impute = ['total_bal_il']


# Impute missing values with 0
df[columns_to_impute] = df[columns_to_impute].fillna(0)

In [ ]:
# After Inputation
columns_to_visualize = ['total_bal_il']

# Plotting settings
plt.figure(figsize=(10, 6))
plt.title("Frequency of Imputed Values")
plt.xlabel("Counts")
plt.ylabel("Frequency")

# Create count plots for each column
for column in columns_to_visualize:
    sns.countplot(data=df, x=column)
    
plt.legend(columns_to_visualize)
plt.show()

il_util

In [ ]:
filtered_df = df.dropna(subset=['il_util'])
total_bal_il_values = filtered_df['il_util'].unique()

# Display unique values and the rest of the columns
filtered_df

In [ ]:
df['il_util'].unique()

In [ ]:
# Check against default_ind
condition = (df['il_util'] > 0) & (df['default_ind'] == 1)
result_df = df[condition]

result_df

In [ ]:
# Before Inputation
columns_to_visualize = ['il_util']

# Plotting settings
plt.figure(figsize=(10, 6))
plt.title("Frequency of Imputed Values")
plt.xlabel("Counts")
plt.ylabel("Frequency")

# Create count plots for each column
for column in columns_to_visualize:
    sns.countplot(data=df, x=column)
    
plt.legend(columns_to_visualize)
plt.show()

In [ ]:
# Inpute with 0

columns_to_impute = ['il_util']


# Impute missing values with 0
df[columns_to_impute] = df[columns_to_impute].fillna(0)

In [ ]:
# After Inputation
columns_to_visualize = ['il_util']

# Plotting settings
plt.figure(figsize=(10, 6))
plt.title("Frequency of Imputed Values")
plt.xlabel("Counts")
plt.ylabel("Frequency")

# Create count plots for each column
for column in columns_to_visualize:
    sns.countplot(data=df, x=column)
    
plt.legend(columns_to_visualize)
plt.show()

open_rv_12m, open_rv_24m

In [ ]:
df[['open_rv_12m', 'open_rv_24m']].describe()

In [ ]:
# Inpute with 0

columns_to_impute = ['open_rv_12m', 'open_rv_24m']

# Impute missing values with 0
df[columns_to_impute] = df[columns_to_impute].fillna(0)

In [ ]:
columns_to_visualize = ['open_rv_12m', 'open_rv_24m']

# Plotting settings
plt.figure(figsize=(10, 6))
plt.title("Frequency of Imputed Values")
plt.xlabel("Counts")
plt.ylabel("Frequency")

# Create count plots for each column
for column in columns_to_visualize:
    sns.countplot(data=df, x=column)
    
plt.legend(columns_to_visualize)
plt.show()

### Generate new feature

In [ ]:
# Generate the new feature
df['open_acc_rate'] = df['open_acc'] / df['total_acc']
# Swap the values and column names
df['open_acc_rate'], df['default_ind'] = df['default_ind'].copy(), df['open_acc_rate'].copy()
df.rename(columns={'open_acc_rate': 'default_ind', 'default_ind': 'open_acc_rate'}, inplace=True)

In [ ]:
df.head()

In [ ]:
np.shape(df)

### Visualize

In [ ]:
# Create a correlation matrix
corr_matrix = df.corr()

# Select the correlation values with 'default_ind'
target_corr = corr_matrix['default_ind']

# Plot the correlation matrix as a heatmap
plt.figure(figsize=(12, 12))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix')
plt.show()

### Attribute ranking

In [ ]:
# Take the absolute values of the correlation
abs_corr = target_corr.abs()

# Sort the absolute correlation values
sorted_corr = abs_corr.sort_values(ascending=False)

# Display the attribute ranking
attribute_ranking = sorted_corr.reset_index()
attribute_ranking.columns = ['Attribute', 'Absolute Correlation']
attribute_ranking